In [15]:
import shared_functions
#下面用到什么定义就从shared_functions中载入
from shared_functions import read_from_files
import os
import pandas as pd
import numpy as np
import math
import sys
import time
import pickle
import json
import datetime
import random
import torch
import sklearn
from sklearn import *
from shared_functions import performance_assessment
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')
sns.set_style('darkgrid', {'axes.facecolor': '0.9'})

import graphviz
import xgboost
# For imbalanced learning
#import imblearn
import warnings
warnings.filterwarnings('ignore')
from shared_functions import training_loop
import math
import torch.nn as nn
from shared_functions import prepare_generators
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [17]:
DIR_INPUT='./data/simulated-data-transformed-main/data/' 

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: user 436 ms, sys: 802 ms, total: 1.24 s
Wall time: 1.24 s
919767 transactions loaded, containing 8195 fraudulent transactions


In [18]:
from shared_functions import get_train_test_set, scaleData
# Set the starting day for the training period, and the deltas
start_date_training = datetime.datetime.strptime("2018-07-25", "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7
delta_valid = delta_test
start_date_training_with_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))
(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training_with_valid,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)
# By default, scales input data
(train_df, valid_df)=scaleData(train_df, valid_df,input_features)

In [19]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

Selected device is cuda


In [20]:
x_train = torch.FloatTensor(train_df[input_features].values)
x_valid = torch.FloatTensor(valid_df[input_features].values)
y_train = torch.FloatTensor(train_df[output_feature].values)
y_valid = torch.FloatTensor(valid_df[output_feature].values)

In [21]:
# new Dataset: FraudDatasetUnsupervised, which only receives the descriptive features of the transaction x
# and returns it as both input and output.去掉标签的数据
class FraudDatasetUnsupervised(torch.utils.data.Dataset):
    
    def __init__(self, x,output=True):
        'Initialization'
        self.x = x
        self.output = output

    def __len__(self):
        'Returns the total number of samples'
        return len(self.x)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample index
        item = self.x[index].to(DEVICE)
        if self.output:
            return item, item
        else:
            return item

In [22]:
training_set = FraudDatasetUnsupervised(x_train)
valid_set = FraudDatasetUnsupervised(x_valid)
training_generator,valid_generator = prepare_generators(training_set, valid_set, batch_size = 64)

In [23]:
def per_sample_mse(model, generator):
    
    model.eval()
    criterion = torch.nn.MSELoss(reduction="none")
    batch_losses = []
    
    for x_batch, y_batch in generator:
        # Forward pass
        y_pred = model(x_batch)
        # Compute Loss
        loss = criterion(y_pred.squeeze(), y_batch)
        loss_app = list(torch.mean(loss,axis=1).detach().cpu().numpy())
        batch_losses.extend(loss_app)
    
    return batch_losses

In [26]:
from tqdm import tqdm
import optuna

class GELU(nn.Module):  # 激活函数含噪声参数
    def __init__(self,sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, input):
        temp = input * (1 + torch.erf(input / torch.sqrt(torch.tensor(2.0)) / self.sigma)) / 2  # 前向传播
        return temp

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.gelu1 = GELU(trial.suggest_float("sigma_1", 0.1, 5.0))
            self.gelu2 = GELU(trial.suggest_float("sigma_2", 0.1, 5.0))
            self.gelu3 = GELU(trial.suggest_float("sigma_3", 0.1, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.gelu1(hidden)
            
            code = self.fc2(hidden)
            code = self.gelu2(code)
 
            hidden = self.fc3(code)
            hidden = self.gelu3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

In [27]:
best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=20)

[I 2024-12-03 14:57:32,856] A new study created in memory with name: no-name-0b4ac85c-daa8-452b-a93e-dee1bdfce4cb



Epoch 0: train loss: 0.5043518998683296
valid loss: 0.1705069158816598

Epoch 1: train loss: 0.11854787085768027
valid loss: 0.06667149253393131

Epoch 2: train loss: 0.048344600192461715
valid loss: 0.03007140745882128

Epoch 3: train loss: 0.02688401435318458
valid loss: 0.02167191175448764

Epoch 4: train loss: 0.019827575727328517
valid loss: 0.015809430936198743

Epoch 5: train loss: 0.01385420200817401
valid loss: 0.01052539900678103

Epoch 6: train loss: 0.008445383857999773
valid loss: 0.005971252251304482

Epoch 7: train loss: 0.006085145780033153
valid loss: 0.004990899246287802

Epoch 8: train loss: 0.005184439288489663
valid loss: 0.004239611204489659

Epoch 9: train loss: 0.004296266963530909
valid loss: 0.003538998474921685

Epoch 10: train loss: 0.0034186265325168393
valid loss: 0.0028726795027948547

Epoch 11: train loss: 0.002834448125102089
valid loss: 0.0025307620523110684

Epoch 12: train loss: 0.0025308152933847194
valid loss: 0.0022701204197145388

Epoch 13: trai

[I 2024-12-03 15:02:38,312] Trial 0 finished with value: 1.078 and parameters: {'sigma_1': 0.14552588024296545, 'sigma_2': 3.205164181715946, 'sigma_3': 3.1080087541916464}. Best is trial 0 with value: 1.078.


0.851
0.227
Updated Best Model!
Confusion Matrix:
[[52539  5592]
 [  115   259]]

Epoch 0: train loss: 0.5100161156423005
valid loss: 0.18631743718691862

Epoch 1: train loss: 0.13537306064672155
valid loss: 0.08576069586182553

Epoch 2: train loss: 0.06901931362743018
valid loss: 0.04214849866763816

Epoch 3: train loss: 0.035337643599917624
valid loss: 0.025354734893705023

Epoch 4: train loss: 0.02413255881952624
valid loss: 0.019735981344850988

Epoch 5: train loss: 0.01918766487667597
valid loss: 0.01509066897500408

Epoch 6: train loss: 0.013653182321767649
valid loss: 0.010111595990512865

Epoch 7: train loss: 0.009358499048234999
valid loss: 0.006357648417027921

Epoch 8: train loss: 0.0060037731054554165
valid loss: 0.004503896329399259

Epoch 9: train loss: 0.004735583234017441
valid loss: 0.0036386730307153337

Epoch 10: train loss: 0.004073142252566852
valid loss: 0.003147451924109231

Epoch 11: train loss: 0.00357803389001335
valid loss: 0.002781221791313035

Epoch 12: tra

[I 2024-12-03 15:07:39,417] Trial 1 finished with value: 1.09 and parameters: {'sigma_1': 2.189625271327292, 'sigma_2': 3.5083552374676574, 'sigma_3': 3.11379397477385}. Best is trial 1 with value: 1.09.


...

0.85
0.226

Epoch 0: train loss: 0.5235756852060386
valid loss: 0.19733124214931916

Epoch 1: train loss: 0.15291965138843822
valid loss: 0.08891574901228394

Epoch 2: train loss: 0.0696506790948558
valid loss: 0.04390825437473469

Epoch 3: train loss: 0.03554722154156855
valid loss: 0.02396262898593327

Epoch 4: train loss: 0.021548273467439198
valid loss: 0.015898322349648334

Epoch 5: train loss: 0.013465137484022211
valid loss: 0.008649133908679415

Epoch 6: train loss: 0.007491650068500288
valid loss: 0.0052961533665249905

Epoch 7: train loss: 0.005419276597514307
valid loss: 0.004097416932450333

Epoch 8: train loss: 0.004431547184811324
valid loss: 0.003381235378697677

Epoch 9: train loss: 0.003679394464780457
valid loss: 0.002717447976831123

Epoch 10: train loss: 0.0028965014587717403
valid loss: 0.002058307253475723

Epoch 11: train loss: 0.002001402418825412
valid loss: 0.0013728112392661878

Epoch 12: train loss: 0.001351132528527184
valid loss: 0.0010235778488666633

Epo

[I 2024-12-03 16:31:10,474] Trial 18 finished with value: 1.074 and parameters: {'sigma_1': 3.042130527859531, 'sigma_2': 2.6223375108832356, 'sigma_3': 2.5743189348242477}. Best is trial 9 with value: 1.093.


0.848
0.226

Epoch 0: train loss: 0.5139273208767232
valid loss: 0.2142406779478808

Epoch 1: train loss: 0.16390869611193984
valid loss: 0.0941314023537714

Epoch 2: train loss: 0.07061060176303977
valid loss: 0.04397370770695757

Epoch 3: train loss: 0.0389184501257157
valid loss: 0.028386448749183307

Epoch 4: train loss: 0.02587354596237693
valid loss: 0.01979564630574053

Epoch 5: train loss: 0.017856765624819533
valid loss: 0.013124493261687254

Epoch 6: train loss: 0.011049431554365895
valid loss: 0.007891007984067248

Epoch 7: train loss: 0.007663663897403368
valid loss: 0.0059879217650537775

Epoch 8: train loss: 0.006230211903316184
valid loss: 0.004885987581079571

Epoch 9: train loss: 0.005254609970955524
valid loss: 0.004098849219035947

Epoch 10: train loss: 0.004500038311932499
valid loss: 0.0035585107000626094

Epoch 11: train loss: 0.0038589565687720266
valid loss: 0.0029683646403731462

Epoch 12: train loss: 0.0031893203871050427
valid loss: 0.002366890944095196

Epoc

[I 2024-12-03 16:36:19,587] Trial 19 finished with value: 1.078 and parameters: {'sigma_1': 2.0920236420400364, 'sigma_2': 3.8386305884146266, 'sigma_3': 3.7504724673749488}. Best is trial 9 with value: 1.093.


0.852
0.226


In [17]:
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: GELU1={sigma1}, GELU2={sigma2}, GELU3={sigma3}")

Best trial parameters:
{'sigma_1': 4.154498525138923, 'sigma_2': 3.9751598714493444, 'sigma_3': 2.6531766740713265}
Optimal sigma values: GELU1=4.154498525138923, GELU2=3.9751598714493444, GELU3=2.6531766740713265


In [22]:
#实验结果：[I 2024-12-02 21:42:22,265] Trial 4 finished with value: 1.1 and parameters: {'sigma_1': 4.108071217704759, 'sigma_2': 3.05488181077701, 'sigma_3': 2.3962617153103962}. Best is trial 4 with value: 1.1.
0.869
0.231

0.231

In [30]:
class SiLU(nn.Module):  # 激活函数含噪声参数
    def __init__(self,sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, input):
        temp = input * torch.sigmoid(input / self.sigma) # 前向传播
        return temp

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.silu1 = SiLU(trial.suggest_float("sigma_1", 0.1, 5.0))
            self.silu2 = SiLU(trial.suggest_float("sigma_2", 0.1, 5.0))
            self.silu3 = SiLU(trial.suggest_float("sigma_3", 0.1, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.silu1(hidden)
            
            code = self.fc2(hidden)
            code = self.silu2(code)
 
            hidden = self.fc3(code)
            hidden = self.silu3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2024-12-03 16:40:25,533] A new study created in memory with name: no-name-d21cbee2-61f4-4e83-a824-4529d8edfe82



Epoch 0: train loss: 0.5186284254118088
valid loss: 0.19544177768660373

Epoch 1: train loss: 0.14958315862638316
valid loss: 0.08783522057728689

Epoch 2: train loss: 0.06400370962889312
valid loss: 0.040033551462279646

Epoch 3: train loss: 0.03564840887030355
valid loss: 0.026923045368427462

Epoch 4: train loss: 0.025361228793018976
valid loss: 0.01981965399301443

Epoch 5: train loss: 0.018033098359992925
valid loss: 0.012378497870099512

Epoch 6: train loss: 0.009803588134165405
valid loss: 0.00604965213299449

Epoch 7: train loss: 0.006159807161390639
valid loss: 0.004686248142022864

Epoch 8: train loss: 0.005104218545747319
valid loss: 0.003964578307264286

Epoch 9: train loss: 0.004480979399778747
valid loss: 0.0035394821730773142

Epoch 10: train loss: 0.004038532590181932
valid loss: 0.003179285231636725

Epoch 11: train loss: 0.0036960203493509023
valid loss: 0.002949897694029932

Epoch 12: train loss: 0.0033915769288772237
valid loss: 0.002642244034156692

Epoch 13: trai

[I 2024-12-03 16:44:41,965] Trial 0 finished with value: 1.071 and parameters: {'sigma_1': 0.5938746851891842, 'sigma_2': 4.593083040719566, 'sigma_3': 1.9238539990433898}. Best is trial 0 with value: 1.071.


0.842
0.229
Updated Best Model!
Confusion Matrix:
[[52541  5590]
 [  113   261]]

Epoch 0: train loss: 0.5137471224592703
valid loss: 0.20596642149764982

Epoch 1: train loss: 0.14696430596549698
valid loss: 0.09430717821818232

Epoch 2: train loss: 0.06777768532779374
valid loss: 0.03924704181854842

Epoch 3: train loss: 0.0348251954817977
valid loss: 0.025650239758234206

Epoch 4: train loss: 0.02258116255975533
valid loss: 0.0160290422793376

Epoch 5: train loss: 0.01437582666353531
valid loss: 0.010615266062054641

Epoch 6: train loss: 0.009687539117489131
valid loss: 0.006520124790487719

Epoch 7: train loss: 0.006243562757217434
valid loss: 0.004570119480648246

Epoch 8: train loss: 0.004862579898588203
valid loss: 0.0036987498808458774

Epoch 9: train loss: 0.003985481673271091
valid loss: 0.003079815643020178

Epoch 10: train loss: 0.00315499121897959
valid loss: 0.0022844470843362314

Epoch 11: train loss: 0.002228877993178577
valid loss: 0.0015906154227066439

Epoch 12: train

[I 2024-12-03 16:48:50,306] Trial 1 finished with value: 1.08 and parameters: {'sigma_1': 1.692766487265329, 'sigma_2': 3.895310850172154, 'sigma_3': 1.1147221505432356}. Best is trial 1 with value: 1.08.


...

0.848
0.229

Epoch 0: train loss: 0.49277501445028565
valid loss: 0.16832245744642665

Epoch 1: train loss: 0.1131709270628016
valid loss: 0.05685712383255932

Epoch 2: train loss: 0.04215664542271234
valid loss: 0.02503033932196638

Epoch 3: train loss: 0.02134808688647119
valid loss: 0.01567669936298021

Epoch 4: train loss: 0.01331857297200902
valid loss: 0.00910092431369945

Epoch 5: train loss: 0.007311970536173259
valid loss: 0.004975790648003582

Epoch 6: train loss: 0.005203355244631535
valid loss: 0.003961101322831799

Epoch 7: train loss: 0.004371208733862113
valid loss: 0.0034644036827344306

Epoch 8: train loss: 0.0036928988637021576
valid loss: 0.0029010103147761907

Epoch 9: train loss: 0.0029482287805514982
valid loss: 0.002164085433421443

Epoch 10: train loss: 0.0021279747826585915
valid loss: 0.0015436741969385311

Epoch 11: train loss: 0.001529491000566596
valid loss: 0.0011943819814951978

Epoch 12: train loss: 0.0012568152418589828
valid loss: 0.001036365679178967


[I 2024-12-03 17:18:32,492] Trial 8 finished with value: 1.079 and parameters: {'sigma_1': 0.5190629567692205, 'sigma_2': 0.7500168767080223, 'sigma_3': 4.018114731826644}. Best is trial 6 with value: 1.091.


0.848
0.231

Epoch 0: train loss: 0.5224251648265135
valid loss: 0.19607449546863473

Epoch 1: train loss: 0.1447618489062866
valid loss: 0.08462191391023782

Epoch 2: train loss: 0.0629626552451466
valid loss: 0.035387741493397074

Epoch 3: train loss: 0.02939738475327624
valid loss: 0.021292895414545886

Epoch 4: train loss: 0.01913722195212033
valid loss: 0.01387747174098355

Epoch 5: train loss: 0.01188048152350972
valid loss: 0.007870656399493987

Epoch 6: train loss: 0.007213252371294605
valid loss: 0.005164600491951235

Epoch 7: train loss: 0.005462408037304195
valid loss: 0.0041144904837042825

Epoch 8: train loss: 0.004600781478262889
valid loss: 0.003551350903310533

Epoch 9: train loss: 0.0040506010795640715
valid loss: 0.003089223498773705

Epoch 10: train loss: 0.0035784287933780176
valid loss: 0.0030432348179025323

Epoch 11: train loss: 0.0030592761925011197
valid loss: 0.0022214195791560687

Epoch 12: train loss: 0.002261947664716254
valid loss: 0.0015274437928770777

E

[I 2024-12-03 17:22:42,547] Trial 9 finished with value: 1.083 and parameters: {'sigma_1': 1.1458862371478755, 'sigma_2': 2.5243479611951534, 'sigma_3': 0.8757207872180713}. Best is trial 6 with value: 1.091.


0.854
0.229
Best trial parameters:
{'sigma_1': 0.32487841546726154, 'sigma_2': 3.034287558059582, 'sigma_3': 2.931114456661248}
Optimal sigma values: SiLU1=0.32487841546726154, SiLU2=3.034287558059582, SiLU3=2.931114456661248


In [ ]:
class ExLU(nn.Module):  # 激活函数含噪声参数
    def __init__(self,sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, input):
        temp = torch.where(input >= 0, input, input * torch.exp(input / self.sigma))
        return temp
    

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.exlu1 = ExLU(trial.suggest_float("sigma_1", 0.01, 3.0))
            self.exlu2 = ExLU(trial.suggest_float("sigma_2", 0.01, 3.0))
            self.exlu3 = ExLU(trial.suggest_float("sigma_3", 0.01, 3.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.exlu1(hidden)
            
            code = self.fc2(hidden)
            code = self.exlu2(code)
 
            hidden = self.fc3(code)
            hidden = self.exlu3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2024-12-03 17:45:19,872] A new study created in memory with name: no-name-3ac2af96-e35a-4c77-a3a4-b9f8a7916e43



Epoch 0: train loss: 0.3569668272926862
valid loss: 0.08408807875191579

Epoch 1: train loss: 0.058444605102089466
valid loss: 0.02983473752080743

Epoch 2: train loss: 0.021477868768028617
valid loss: 0.010987815534176885

Epoch 3: train loss: 0.008158337876661885
valid loss: 0.004805943643155157

Epoch 4: train loss: 0.004319938753346911
valid loss: 0.002776533696556189

Epoch 5: train loss: 0.00255721751130836
valid loss: 0.0018168622872135679

Epoch 6: train loss: 0.0017774406100229767
valid loss: 0.001360740852058748

Epoch 7: train loss: 0.0013616500734765595
valid loss: 0.0010592044986367144

Epoch 8: train loss: 0.0010919464849127923
valid loss: 0.0008741804301583519

Epoch 9: train loss: 0.0009028532413574805
valid loss: 0.0007549264987350488

Epoch 10: train loss: 0.0007720212189455732
valid loss: 0.0006235373542323464

Epoch 11: train loss: 0.0006681349488582024
valid loss: 0.0005617792065741361

Epoch 12: train loss: 0.0005821138573075907
valid loss: 0.0004740447315668606


[I 2024-12-03 17:49:58,501] Trial 0 finished with value: 1.055 and parameters: {'sigma_1': 2.907351357558916, 'sigma_2': 2.3005796055608103, 'sigma_3': 1.647378505613268}. Best is trial 0 with value: 1.055.


0.838
0.217
Updated Best Model!
Confusion Matrix:
[[52534  5597]
 [  120   254]]

Epoch 0: train loss: nan
valid loss: nan

Epoch 1: train loss: nan
valid loss: nan

Epoch 2: train loss: nan
valid loss: nan

Epoch 3: train loss: nan
valid loss: nan

Epoch 4: train loss: nan
valid loss: nan

Epoch 5: train loss: nan
valid loss: nan

Epoch 6: train loss: nan
valid loss: nan

Epoch 7: train loss: nan
valid loss: nan

Epoch 8: train loss: nan
valid loss: nan

Epoch 9: train loss: nan
valid loss: nan

Epoch 10: train loss: nan
valid loss: nan

Epoch 11: train loss: nan
valid loss: nan

Epoch 12: train loss: nan
valid loss: nan

Epoch 13: train loss: nan
valid loss: nan

Epoch 14: train loss: nan
valid loss: nan

Epoch 15: train loss: nan
valid loss: nan
